In [1]:
API_KEY = "QopJK73RwNkwj5hfFk1cInN8BDM0pcaVC9hWYXqU"
TRIPS_URL = "https://valladolid.publicbikesystem.net/customer/v3.0/profile/trips?count=100"
STATIONS_URL = "https://valladolid.publicbikesystem.net/customer/gbfs/v2/en/station_information"

In [11]:
import requests
import sqlite3

con = sqlite3.connect("biki.db")
cur = con.cursor()

In [12]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS stations (
      id INTEGER UNIQUE PRIMARY KEY,
      name TEXT,
      lat REAL,
      lon REAL,
      address TEXT,
      obcn TEXT,
      capacity INTEGER)""")

In [13]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS trips (
      id INTEGER UNIQUE PRIMARY KEY,
      bikeId INTEGER,
      startTime TEXT,
      startStation INTEGER,
      endTime TEXT,
      endStation INTEGER,
      duration INTEGER,
      price INTEGER,
      bikeObcn TEXT,
      bikeModel TEXT,
      bikeMsnbc TEXT,
      bikeQrCode TEXT,
      FOREIGN KEY(startStation) REFERENCES stations(id),
      FOREIGN KEY(endStation) REFERENCES stations(id))""")

In [30]:
response = requests.get("https://valladolid.publicbikesystem.net/customer/gbfs/v2/en/station_information")
stations_data = response.json()["data"]["stations"]
cur.executemany("""
    INSERT OR IGNORE INTO stations
    VALUES (:station_id, :name, :lat, :lon, :address, :obcn, :capacity)""", stations_data)
con.commit()

In [1]:
user = input("Usuario de BIKI")
password = input("Contraseña de BIKI")

response = requests.get("https://valladolid.publicbikesystem.net/customer/v3.0/profile/trips?count=100", headers={"X-API-KEY": API_KEY}, auth=(user, password))

Usuario de BIKI X
Contraseña de BIKI X


NameError: name 'requests' is not defined

In [31]:
def map_trip(orig):
    return {
        "id": orig["tripId"],
        "bikeId": orig["bikeId"],
        "startTime": orig["startTime"],
        "startStationId": orig["startStationId"],
        "endTime": orig["endTime"],
        "endStationId": orig["endStationId"],
        "duration": orig["duration"],
        "price": int(orig["price"] * 100),
        "bikeQrCode": orig["bikeQrCode"],
        "bikeModel": orig["bike"]["model"],
        "bikeObcn": orig["bike"]["obcn"],
        "bikeMsnbc": orig["bike"]["msnbc"],
    }

trips = map(map_trip, response.json()["trips"])
cur.executemany("""
    INSERT OR IGNORE INTO trips
    VALUES (:id, :bikeId, :startTime, :startStationId, :endTime, :endStationId, :duration, :price, :bikeObcn, :bikeModel, :bikeMsnbc, :bikeQrCode)""", trips)
con.commit()

KeyError: 'trips'